# 使用Trainer和Tester快速训练和测试

## 数据读入和处理

In [1]:
from fastNLP.io import SST2Pipe

pipe = SST2Pipe()
databundle = pipe.process_from_file()
vocab = databundle.get_vocab('words')
print(databundle)
print(databundle.get_dataset('train')[0])
print(databundle.get_vocab('words'))

/remote-home/ynzheng/anaconda3/envs/now/lib/python3.8/site-packages/FastNLP-0.5.0-py3.8.egg/fastNLP/io/loader/classification.py:340: UserWarning: SST2's test file has no target.


In total 3 datasets:
	test has 1821 instances.
	train has 67349 instances.
	dev has 872 instances.
In total 2 vocabs:
	words has 16292 entries.
	target has 2 entries.

+-----------------------------------+--------+-----------------------------------+---------+
| raw_words                         | target | words                             | seq_len |
+-----------------------------------+--------+-----------------------------------+---------+
| hide new secretions from the p... | 1      | [4110, 97, 12009, 39, 2, 6843,... | 7       |
+-----------------------------------+--------+-----------------------------------+---------+
Vocabulary(['hide', 'new', 'secretions', 'from', 'the']...)


In [2]:
train_data = databundle.get_dataset('train')[:5000]
train_data, test_data = train_data.split(0.015)
dev_data = databundle.get_dataset('dev')
print(len(train_data),len(dev_data),len(test_data))

4925 872 75


In [3]:
train_data.print_field_meta()

+-------------+-----------+--------+-------+---------+
| field_names | raw_words | target | words | seq_len |
+-------------+-----------+--------+-------+---------+
|   is_input  |   False   | False  |  True |   True  |
|  is_target  |   False   |  True  | False |  False  |
| ignore_type |           | False  | False |  False  |
|  pad_value  |           |   0    |   0   |    0    |
+-------------+-----------+--------+-------+---------+


## 使用内置模型训练

In [4]:
from fastNLP.models import CNNText

#词嵌入的维度
EMBED_DIM = 100

#使用CNNText的时候第一个参数输入一个tuple,作为模型定义embedding的参数
#还可以传入 kernel_nums, kernel_sizes, padding, dropout的自定义值
model_cnn = CNNText((len(vocab),EMBED_DIM), num_classes=2, dropout=0.1)

In [5]:
from fastNLP import AccuracyMetric
from fastNLP import Const

# metrics=AccuracyMetric() 在本例中与下面这行代码等价
metrics=AccuracyMetric(pred=Const.OUTPUT, target=Const.TARGET)

In [6]:
from fastNLP import CrossEntropyLoss

# loss = CrossEntropyLoss() 在本例中与下面这行代码等价
loss = CrossEntropyLoss(pred=Const.OUTPUT, target=Const.TARGET)

In [7]:
# 这表示构建了一个损失函数类，由func计算损失函数，其中将从模型返回值或者DataSet的target=True的field
# 当中找到一个参数名为`pred`的参数传入func一个参数名为`input`的参数；找到一个参数名为`label`的参数
# 传入func作为一个名为`target`的参数
#下面自己构建了一个交叉熵函数，和之后直接使用fastNLP中的交叉熵函数是一个效果
import torch
from fastNLP import LossFunc
func = torch.nn.functional.cross_entropy
loss_func = LossFunc(func, input=Const.OUTPUT, target=Const.TARGET)

In [8]:
import torch.optim as optim

#使用 torch.optim 定义优化器
optimizer=optim.RMSprop(model_cnn.parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)

In [9]:
from fastNLP import Trainer

#训练的轮数和batch size
N_EPOCHS = 10
BATCH_SIZE = 16

#如果在定义trainer的时候没有传入optimizer参数，模型默认的优化器为torch.optim.Adam且learning rate为lr=4e-3
#这里只使用了loss作为损失函数输入，感兴趣可以尝试其他损失函数（如之前自定义的loss_func）作为输入
trainer = Trainer(model=model_cnn, train_data=train_data, dev_data=dev_data, loss=loss, metrics=metrics,
optimizer=optimizer,n_epochs=N_EPOCHS, batch_size=BATCH_SIZE)
trainer.train()

input fields after batch(if batch size is 2):
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 4]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2020-02-27-11-31-25


Evaluate data in 0.75 seconds!
Evaluation on dev at Epoch 1/10. Step:308/3080: 
AccuracyMetric: acc=0.751147



Evaluate data in 0.83 seconds!
Evaluation on dev at Epoch 2/10. Step:616/3080: 
AccuracyMetric: acc=0.755734



Evaluate data in 1.32 seconds!
Evaluation on dev at Epoch 3/10. Step:924/3080: 
AccuracyMetric: acc=0.758028



Evaluate data in 0.88 seconds!
Evaluation on dev at Epoch 4/10. Step:1232/3080: 
AccuracyMetric: acc=0.741972



...

Evaluate data in 1.04 seconds!
Evaluation on dev at Epoch 7/10. Step:2156/3080: 
AccuracyMetric: acc=0.732798



Evaluate data in 0.57 seconds!
Evaluation on dev at Epoch 8/10. Step:2464/3080: 
AccuracyMetric: acc=0.747706



Evaluate data in 0.48 seconds!
Evaluation on dev at Epoch 9/10. Step:2772/3080: 
AccuracyMetric: acc=0.732798



Evaluate data in 0.48 seconds!
Evaluation on dev at Epoch 10/10. Step:3080/3080: 
AccuracyMetric: acc=0.740826


In Epoch:3/Step:924, got best dev performance:
AccuracyMetric: acc=0.758028
Reloaded the best model.


{'best_eval': {'AccuracyMetric': {'acc': 0.758028}},
 'best_epoch': 3,
 'best_step': 924,
 'seconds': 160.58}

In [10]:
from fastNLP import Tester

tester = Tester(test_data, model_cnn, metrics=AccuracyMetric())
tester.test()

Evaluate data in 0.43 seconds!
[tester] 
AccuracyMetric: acc=0.773333


{'AccuracyMetric': {'acc': 0.773333}}